1. Überprüfen ob "Description" und "Text" der Artikel deckungsgleich sind und gegebenenfalls die betroffenen Artikel entfernen.
2. Nicht-lateinische Schrift entfernen und prüfen ob Heuristiken zur gezielten Entfernung von nicht-deutscher Sprache gefunden werden können (z.B. Originaltitel von Filmen in Klammern hinter dem Artikelnamen)
3. Mindestlänge von Artikeln auf 100 Wörter und Maximallänge auf 2000 Wörter festlegen. Artikel die weniger als 100 Wörter enthalten, werden entfernt und durch längere ersetzt. Artikel mit mehr als 2000 Wörtern werden an der Obergrenze abgeschnitten.
4. Überlappungen zwischen den verschiedenen Datensätzen vor dem Merge prüfen.

In [1]:
import pandas as pd
import regex

%matplotlib inline

In [2]:
categories = pd.read_csv("../data/classification-corpus/final-corpus.csv")
categories.head(5)

,category,description,text,length
0,Herrscher,Adaloald auch Adalwald Adulubaldus 602 in Monz...,Adaloald auch Adalwald Adulubaldus 602 in Monz...,236.0
1,Herrscher,Ælfwine auch Aelfwine Elfwine Aelfuini Aelfwin...,Ælfwine auch Aelfwine Elfwine Aelfuini Aelfwin...,279.0
2,Herrscher,Æscwine auch Aescwine Escuuine 676 war von 674...,Æscwine auch Aescwine Escuuine 676 war von 674...,135.0
3,Herrscher,Æthelberht auch Æþelbryht Æþelbyrht Aedilberct...,Æthelberht auch Æþelbryht Æþelbyrht Aedilberct...,841.0
4,Herrscher,Æthelburg auch Æthelburh Ethelburga Æthelburga...,Æthelburg auch Æthelburh Ethelburga Æthelburga...,303.0


### 1. description == text

In [23]:
categories = categories[categories["description"] != categories["text"]].reset_index(drop=True)

### 2. Nicht-lateinische Schrift

In [24]:
token_pattern = regex.compile(r"\p{Latin}+\p{Connector_Punctuation}?\p{Latin}+|\p{Number}+")

In [25]:
example = """Deutsch: lateinisches Alphabet, 
             Amharisch: የላቲን አልፋቤት,
             Bengalisch: লাতিন বর্ণমালা,
             Arabisch: اَللُّغَةُ اَلْعَرَبِيَّة,
             Chinesisch: 漢字,
             Japanisch: 日本語"""
token_pattern.findall(example)

['Deutsch',
 'lateinisches',
 'Alphabet',
 'Amharisch',
 'Bengalisch',
 'Arabisch',
 'Chinesisch',
 'Japanisch']

In [26]:
def drop_nonlatin(articles):
    return articles.apply(lambda article: " ".join(token_pattern.findall(str(article))))

In [27]:
categories["description"] = drop_nonlatin(categories["description"])
categories["text"] = drop_nonlatin(categories["text"])

### 3. Mindest- bzw. Maximallänge

In [28]:
minimum = 100
maximum = 2000

In [29]:
length = list()

for article in categories["text"]:
    tokens = token_pattern.findall(article)
    length.append(len(tokens))

categories["length"] = length

In [30]:
categories.head(5)

,category,description,text,length
0,Herrscher,Adaloald auch Adalwald Adulubaldus 602 in Monz...,Adaloald auch Adalwald Adulubaldus 602 in Monz...,236
1,Herrscher,Ælfwine auch Aelfwine Elfwine Aelfuini Aelfwin...,Ælfwine auch Aelfwine Elfwine Aelfuini Aelfwin...,279
2,Herrscher,Æscwine auch Aescwine Escuuine 676 war von 674...,Æscwine auch Aescwine Escuuine 676 war von 674...,135
3,Herrscher,Æthelberht auch Æþelbryht Æþelbyrht Aedilberct...,Æthelberht auch Æþelbryht Æþelbyrht Aedilberct...,841
4,Herrscher,Æthelburg auch Æthelburh Ethelburga Æthelburga...,Æthelburg auch Æthelburh Ethelburga Æthelburga...,303


In [31]:
categories = categories[categories["length"] >= minimum].reset_index(drop=True)

In [32]:
cut = categories[categories["length"] > maximum]

In [33]:
for row, article in cut.iterrows():
    tokens = token_pattern.findall(article["text"])
    categories.iloc[row, 2] = " ".join(tokens[:maximum])

In [34]:
length = list()

for article in categories["text"]:
    tokens = token_pattern.findall(article)
    length.append(len(tokens))

categories["length"] = length

In [35]:
categories = categories.drop_duplicates("description")

In [52]:
categories["category"].value_counts()

Gemälde_(20._Jahrhundert)          200
Film                               200
Verein (Bonn)                      200
Chemikaliengruppe                  200
Literatur                          200
Literaturwissenschaft              200
Frauenfußball                      200
Einzelsprache                      200
Millionenstadt                     200
Bekleidungsmarke                   200
Filmschauspieler                   200
Musik                              200
Pkw-Modell                         200
Rockband                           200
Informationssendung (Fernsehen)    199
Western                            199
Computerspiele                     199
Fernsehserien                      198
Burgen                             193
Kriege                             192
Romantik (Literatur)               190
Herrscher                          187
Xbox-Spiel                         186
Krankheit                          184
Fantasy-Fernsehserie               183
Kreditgeschäft           